In [40]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms

In [41]:
import openpyxl
from openpyxl import load_workbook
import pandas as pd
import numpy as np

In [42]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [43]:
device

device(type='cpu')

In [44]:
def load_data(sheets_in_use):
    wb = load_workbook("data1.xlsx")
    sheets = wb.sheetnames
    
    raw_data = []
    c = [(1, 1), (1, 2), (2, 1), (2, 2), (3, 1), (3, 2), ('Log_Facies', 'Fluid')]
    for i, sname in enumerate(sheets):
        print(i, sname)
        if sname not in sheets_in_use:
            continue
        for row in wb[sname]:
            data = [c.value for c in row]
            lf = data[-3]
            f = data[-1]
            if lf is None or f is None:
                continue
            data.append(c.index((lf, f)))
            raw_data.append(data)
    head = raw_data[0]
    head[-1] = "class"
    body = [r for r in raw_data if (r[0]!= head[0] and r[0] is not None)]
    df = pd.DataFrame(body, columns=head)
    return df

In [45]:
data_a = load_data(['1K-01'])
data_b = load_data(['1X-02'])

0 1D-02
1 1F-11
2 1K-01
3 1X-02
4 98_6-8
5 Well_1
0 1D-02
1 1F-11
2 1K-01
3 1X-02
4 98_6-8
5 Well_1


In [46]:
X = data_a[["DEPTH", "CALI", "DT", "GR", "LLD", "LLS", "MSFL", "NPHI", "RHOB"]]
X_2 = data_b[["DEPTH", "CALI", "DT", "GR", "LLD", "LLS", "MSFL", "NPHI", "RHOB"]]

In [47]:
class NeuralNet(nn.Module):
    
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.act = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        return self.fc2(x)

In [48]:
Y = data_a["Log_Facies"]
Y_2 = data_b["Log_Facies"]

In [49]:
input_size = 9
hidden_size = 10
num_classes = 3
epochs = 100
lr = 0.01

In [50]:
model = NeuralNet(input_size, hidden_size, num_classes)

In [51]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
# optimizer = torch.optim.Adam(model.parameters(), lr=lr)



In [52]:
from sklearn.preprocessing import StandardScaler
ss =  StandardScaler()
X = ss.fit_transform(X.values)
X = torch.Tensor(X)
X.shape

X_2 = ss.fit_transform(X_2.values)
X_2 = torch.Tensor(X_2)
X_2.shape

torch.Size([1094, 9])

In [53]:
Y = Y-1
Y = torch.Tensor(Y)
Y = Y.long()

Y_2 = Y_2-1
Y_2 = torch.Tensor(Y_2)
Y_2 = Y_2.long()

In [54]:
for e in range(1000):
    l = 0
    x = X
    label = Y
    pred = model(x)
#     print(pred.shape, label.shape, x.shape)
    loss = criterion(pred, label)
#     print(loss)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
#     for i in range(X.shape[0]):
#         x = X[i].reshape(-1, 9)
#         print(x.shape)
# #         label = torch.Tensor(Y[i])
#         label = Y[i].view(1)
# #         print(Y[i], label)
#         pred = model(x)
    
# #         print(x.shape, label.shape, pred.shape)
#         loss = criterion(pred, label)
#         l += loss
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#     print(e, l)

In [55]:
with torch.no_grad():
    correct = 0
    all_p = []
    all_pred = []
    for i in range(X_2.shape[0]):
        x = X_2[i].reshape(-1, 9)
    #         label = torch.Tensor(Y[i])
#         print(x)
        label = Y_2[i].view(1)
    #         print(Y[i], label)
        pred = model(x)
        all_p.append(pred)
        _, pred = torch.max(pred, 1)
        all_pred.append(pred)
        if pred == label:
            correct += 1
    total = X_2.shape[0]
    print(correct, total, correct/total)

824 1094 0.753199268738574


In [56]:
with torch.no_grad():
    correct = 0
    all_p = []
    all_pred = []
    for i in range(X.shape[0]):
        x = X[i].reshape(-1, 9)
    #         label = torch.Tensor(Y[i])
#         print(x)
        label = Y[i].view(1)
    #         print(Y[i], label)
        pred = model(x)
        all_p.append(pred)
        _, pred = torch.max(pred, 1)
        all_pred.append(pred)
        if pred == label:
            correct += 1
    total = X.shape[0]
    print(correct, total, correct/total)

763 1025 0.744390243902439


In [57]:
all_pred = torch.cat(all_pred)

In [58]:
pd.Series(all_pred.numpy()).value_counts()

0    626
1    399
dtype: int64

In [59]:

Y = data_a["Log_Facies"]


# Six

In [60]:
Y_a = data_a['class']

In [61]:
Y_a.value_counts()

1    426
3    278
2    153
0    104
4     64
Name: class, dtype: int64

In [62]:
X = data_a[["DEPTH", "CALI", "DT", "GR", "LLD", "LLS", "MSFL", "NPHI", "RHOB"]]
X_2 = data_b[["DEPTH", "CALI", "DT", "GR", "LLD", "LLS", "MSFL", "NPHI", "RHOB"]]

In [63]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
X = ss.fit_transform(X.values)
X = torch.Tensor(X)
X.shape

X_2 = ss.fit_transform(X_2.values)
X_2 = torch.Tensor(X_2)
X_2.shape

torch.Size([1094, 9])

In [64]:
Y = data_a["class"]
Y_2 = data_b["class"]
Y = Y
Y = torch.Tensor(Y)
Y = Y.long()

Y_2 = Y_2
Y_2 = torch.Tensor(Y_2)
Y_2 = Y_2.long()

In [65]:
input_size = 9
hidden_size = 100
num_classes = 6
epochs = 100
lr = 0.01

In [66]:
model = NeuralNet(input_size, hidden_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [67]:
for e in range(epochs):
    l = 0
    x = X
    label = Y
    pred = model(x)
#     print(pred.shape, label.shape, x.shape)
    loss = criterion(pred, label)
#     print(loss)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [68]:
with torch.no_grad():
    correct = 0
    all_p = []
    all_pred = []
    for i in range(X_2.shape[0]):
        x = X_2[i].reshape(-1, 9)
    #         label = torch.Tensor(Y[i])
#         print(x)
        label = Y_2[i].view(1)
    #         print(Y[i], label)
        pred = model(x)
        all_p.append(pred)
        _, pred = torch.max(pred, 1)
        all_pred.append(pred)
        if pred == label:
            correct += 1
    total = X_2.shape[0]
    print(correct, total, correct/total)

745 1094 0.6809872029250457


In [69]:
with torch.no_grad():
    correct = 0
    all_p = []
    all_pred = []
    for i in range(X.shape[0]):
        x = X[i].reshape(-1, 9)
    #         label = torch.Tensor(Y[i])
#         print(x)
        label = Y[i].view(1)
    #         print(Y[i], label)
        pred = model(x)
        all_p.append(pred)
        _, pred = torch.max(pred, 1)
        all_pred.append(pred)
        if pred == label:
            correct += 1
    total = X.shape[0]
    print(correct, total, correct/total)

658 1025 0.6419512195121951
